In [ ]:
!pip install torch transformers datasets accelerate peft bitsandbytes


In [ ]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="/teamspace/studios/this_studio/dataset_train_2.json")

In [ ]:
from huggingface_hub import login

#login()


In [ ]:
from transformers import AutoTokenizer

# Carica il tokenizer pre-addestrato del modello Llama 2 7B in formato Hugging Face.
# Il tokenizer serve a convertire testo in token numerici comprensibili dal modello.
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")

# Imposta il token di padding uguale al token di fine sequenza (EOS token).
# Questo è utile perché alcuni modelli non hanno un token di padding dedicato.
tokenizer.pad_token = tokenizer.eos_token


# Funzione per tokenizzare un esempio del dataset
def tokenize(example):
    # Crea un prompt strutturato a partire dai campi del dataset
    # 'instruction', 'input', 'output'. Il prompt ha la forma:
    # ### Instruction:
    # <istruzione>
    # ### Input:
    # <input>
    # ### Response:
    # <output>
    # Questo formato è spesso usato per addestrare modelli instruction-following.
    prompt = f"### Instruction:\n{example['instruction']}\n### Input:\n{example['text']}\n### Response:\n{example['label']}"
    
    # Tokenizza il prompt:
    # - truncation=True → tronca il testo se supera max_length
    # - padding="max_length" → aggiunge padding fino a max_length
    # - max_length=1024 → lunghezza massima dei token
    # Restituisce un dizionario con input_ids e attention_mask pronto per il modello.
    return tokenizer(prompt, truncation=True, padding="max_length", max_length=1024)


# Applica la funzione di tokenizzazione a tutto il dataset.
# `dataset.map()` crea un nuovo dataset in cui ogni esempio è già tokenizzato.
tokenized_dataset = dataset.map(tokenize)


In [ ]:
from transformers import AutoModelForCausalLM
from peft import get_peft_model, LoraConfig, TaskType

# Carica il modello Llama-2 7B pre-addestrato in modalità Causal Language Modeling.
# load_in_8bit=True → utilizza la quantizzazione a 8 bit per ridurre l'uso di memoria.
# device_map="auto" → assegna automaticamente i layer del modello ai dispositivi disponibili (CPU/GPU).
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-hf",
    load_in_8bit=True,
    device_map="auto"
)

# Configurazione per il LoRA (Low-Rank Adaptation)
# LoRA permette di fare fine-tuning aggiungendo pochi parametri senza aggiornare tutto il modello.
peft_config = LoraConfig(
    r=8,                        # Rank della matrice di aggiornamento low-rank
    lora_alpha=32,               # Moltiplicatore di scaling per stabilizzare l’addestramento LoRA
    target_modules=["q_proj", "v_proj"],  # Layer del modello dove applicare LoRA (tipicamente Q e V delle attention)
    lora_dropout=0.05,           # Dropout applicato ai pesi LoRA per regolarizzazione
    bias="none",                 # Non aggiunge bias aggiuntivo nel fine-tuning
    task_type=TaskType.CAUSAL_LM # Tipo di task: Causal Language Modeling
)

# Applica LoRA al modello originale.
# Restituisce un modello PEFT che contiene i pesi originali congelati + i parametri LoRA addestrabili.
model = get_peft_model(model, peft_config)


In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

# Configurazione dei parametri di training
training_args = TrainingArguments(
    output_dir="./llama-finetuned",  # Cartella dove salvare i checkpoint del modello fine-tuned
    per_device_train_batch_size=2,    # Dimensione del batch per GPU/TPU/CPU
    gradient_accumulation_steps=4,    # Accumula i gradienti per simulare batch più grandi
    num_train_epochs=1,               # Numero di epoche di training sul dataset
    learning_rate=2e-4,               # Learning rate per l'ottimizzatore
    logging_dir="./logs",             # Cartella per salvare i log di training
    logging_steps=10,                 # Frequenza (in step) di scrittura dei log
    save_strategy="epoch",            # Salva il modello alla fine di ogni epoca
    fp16=True                         # Abilita mixed precision (half precision) per ridurre uso memoria e velocizzare training
)

# Collator dei dati per il Language Modeling
# Prepara batch di input per il modello
# mlm=False → modello non usa masked language modeling, adatto a causal LM come Llama
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

# Creazione del Trainer di Hugging Face
trainer = Trainer(
    model=model,                      # Modello da allenare
    args=training_args,                # Parametri di training definiti sopra
    train_dataset=tokenized_dataset["train"],  # Dataset di training già tokenizzato
    tokenizer=tokenizer,               # Tokenizer associato al modello
    data_collator=data_collator        # Funzione che prepara i batch durante il training
)

# Avvia il training del modello
trainer.train()


In [ ]:
prompt = """### Instruction:
You are a classification model specializing in emails, and your job is to detect phishing: respond only with \"0\" if it is not phishing or \"1\" if it is phishing, without explanations, symbols, additional letters, or other characters.
### Input:
We attempted to deliver your package today, but were unable to complete the delivery due to missing address information.
Please update your delivery details as soon as possible to avoid return of the shipment:
Update Delivery Information
Thank you for your cooperation,
Logistics Service Team
### Response:
"""

enc = tokenizer(                                               # Tokenizza il prompt usando il tokenizer del modello
    prompt,                                                    # Testo da tokenizzare
    return_tensors="pt",                                       # Richiede tensori PyTorch come output
    padding=True,                                              # Applica padding automatico
)

input_ids = enc.input_ids.cuda()                               # Sposta gli input IDs sulla GPU
attention_mask = enc.attention_mask.cuda()                     # Sposta la attention mask sulla GPU

outputs = model.generate(                                      # Genera l’output del modello
    input_ids=input_ids,                                       # Fornisce i token di input
    attention_mask=attention_mask,                             # Fornisce la maschera di attenzione
    max_new_tokens=1,                                          # Limita l’output a un solo token (0 o 1)
    pad_token_id=tokenizer.eos_token_id                        # Specifica il token di padding
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))  # Decodifica e stampa l’output finale del modello

In [ ]:
model.save_pretrained("./llama-finetuned")
tokenizer.save_pretrained("./llama-finetuned")

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# ----------------------------
# CONFIGURAZIONE PATH E MODELLO
# ----------------------------

BASE_MODEL_NAME = "meta-llama/Llama-2-7b-hf"
LORA_ADAPTER_PATH = "./llama-finetuned"

# ----------------------------
# CARICAMENTO TOKENIZER
# ----------------------------

tokenizer = AutoTokenizer.from_pretrained(LORA_ADAPTER_PATH)

# Per LLaMA è buona pratica assicurarsi che il pad_token sia definito
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# ----------------------------
# CARICAMENTO MODELLO BASE
# ----------------------------

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_NAME,
    load_in_8bit=True,        # oppure load_in_4bit=True se usi QLoRA
    device_map="auto"
)

# ----------------------------
# CARICAMENTO ADAPTER LoRA
# ----------------------------

model = PeftModel.from_pretrained(
    base_model,
    LORA_ADAPTER_PATH
)

model.eval()



In [ ]:
def dataset_metrics(df, name="Dataset"):          # Definisce una funzione che calcola e stampa metriche descrittive di un DataFrame
    print(f"\n📊 Metriche - {name}")               # Stampa il titolo delle metriche, includendo il nome del dataset
    print("-" * 50)                               # Stampa una linea separatrice lunga 50 caratteri
    print(f"Numero righe: {df.shape[0]}")          # Stampa il numero di righe del DataFrame
    print(f"Numero colonne: {df.shape[1]}")       # Stampa il numero di colonne del DataFrame
    print("\nColonne:")                            # Stampa l’intestazione della sezione colonne
    print(df.columns.tolist())                    # Stampa la lista dei nomi delle colonne
    print("\nValori mancanti per colonna:")       # Stampa l’intestazione della sezione sui valori mancanti
    print(df.isnull().sum())                      # Calcola e stampa il numero di valori nulli per ciascuna colonna
    print("\nTipi di dato:")                      # Stampa l’intestazione della sezione sui tipi di dato
    print(df.dtypes)                              # Stampa il tipo di dato associato a ogni colonna


In [ ]:
import pandas as pd                                  # Importa la libreria pandas per la gestione e l’analisi dei dati
import csv                                           # Importa il modulo csv (utile per operazioni su file CSV)
import sys                                           # Importa il modulo sys per interazioni con il sistema (argomenti, I/O, ecc.)

df = pd.read_json("dataset_test.json")               # Carica il file JSON in un DataFrame pandas

df = df.rename(columns={"text": "body"})             # Rinomina la colonna 'text' in 'body' per uniformare lo schema del dataset

dataset_metrics(df, "Dataset Di Test con metadata")  # Chiama la funzione dataset_metrics per stampare le metriche del dataset


In [ ]:
# Stampa informazioni iniziali
print("Numero di righe iniziali:", len(df))              # Stampa il numero totale di righe del DataFrame originale
print("Conteggio label iniziali:")                       # Stampa l’intestazione per il conteggio delle etichette
print(df['label'].value_counts())                        # Calcola e stampa la frequenza di ciascun valore della colonna 'label'

# Rimuovi righe con label o body vuoti o NaN
df_clean = df.dropna(subset=['label', 'body'])           # Rimuove le righe con valori NaN nelle colonne 'label' o 'body'
df_clean = df_clean[df_clean['body'].str.strip() != ''] # Filtra le righe in cui 'body' non è una stringa vuota o solo spazi

# Stampa informazioni dopo pulizia
print("\nNumero di righe dopo la pulizia:", len(df_clean)) # Stampa il numero di righe dopo l’operazione di pulizia
print("Conteggio label dopo la pulizia:")                 # Stampa l’intestazione per il conteggio delle etichette post-pulizia
print(df_clean['label'].value_counts())                   # Calcola e stampa la frequenza delle etichette nel dataset pulito


In [ ]:
risposte = []                                   # Lista per memorizzare le predizioni del modello
risposte_vere = []                              # Lista per memorizzare le label reali (ground truth)

corretti = 0                                   # Contatore delle predizioni corrette
righe_valide = 0                               # Contatore delle righe effettivamente valutate

dim = len(df)                                  # Numero totale di righe del DataFrame

for i in range(0, dim):                        # Cicla su tutte le righe del DataFrame
    first_row = df.iloc[i]                     # Estrae la riga i-esima come Series

    # Estrai body e label separatamente
    body_text = first_row['body']              # Testo dell’email da classificare
    label = first_row['label']                 # Label attesa (0 o 1)

    #print("Body:\n", body_text, "TIPO: ", type(body_text))
    #print("\nLabel Attesa:\n", label_text, "TIPO: ", type(label_text))
    # Commenti di debug (attualmente disabilitati)

    prompt = f"""                              # Costruisce dinamicamente il prompt per il modello
### Instruction:
You are a classification model specializing in emails, and your job is to detect phishing: respond only with \"0\" if it is not phishing or \"1\" if it is phishing, without explanations, symbols, additional letters, or other characters.
### Input:
{body_text}                                    # Inserisce il testo dell’email nel prompt
### Response:
"""

    #print(prompt)
    # Debug opzionale per visualizzare il prompt completo

    enc = tokenizer(                           # Tokenizza il prompt
        prompt,
        return_tensors="pt",                   # Restituisce tensori PyTorch
        padding=True,                          # Applica padding automatico
    )

    num_tokens = enc.input_ids.shape[1]        # Calcola il numero totale di token del prompt

    if num_tokens > 1024:                       # Controlla il limite massimo di token consentiti
        # Salta questa riga
        continue                               # Esclude l’email troppo lunga dalla valutazione

    righe_valide += 1                          # Incrementa il contatore delle righe valide

    input_ids = enc.input_ids.cuda()           # Sposta gli input IDs sulla GPU
    attention_mask = enc.attention_mask.cuda() # Sposta la attention mask sulla GPU

    outputs = model.generate(                  # Genera la risposta del modello
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=1,                      # Limita l’output a un solo token (0 o 1)
        pad_token_id=tokenizer.eos_token_id    # Specifica il token di padding
    )
    
    risposta = tokenizer.decode(               # Decodifica l’output del modello in testo
        outputs[0], 
        skip_special_tokens=True
    )

    risultato = risposta.split(                # Estrae la parte successiva a "### Response:"
        "### Response:", 
        1
    )[1].strip()

    try:
        risultato = float(risultato)           # Converte la risposta in valore numerico

        if abs(risultato - label) <= 0.1:      # Verifica se la predizione coincide con la label
            corretti = corretti + 1             # Incrementa il contatore dei corretti
    except:
        risultato = "Formato Sbagliato"        # Gestisce output non numerici o malformati

    risposte.append(risultato)                 # Aggiunge la predizione alla lista delle risposte
    risposte_vere.append(label)                # Aggiunge la label reale alla lista di riferimento

    if righe_valide % 100 == 0:                # Ogni 100 righe valide, stampa feedback parziale
        accuracy_temp = (
            corretti / righe_valide 
            if righe_valide > 0 else 0
        )                                      # Calcola l’accuracy parziale
        formati_sbagliati = risposte.count(
            "Formato Sbagliato"
        )                                      # Conta le risposte con formato errato

        print(
            f"[FEEDBACK PARZIALE] "
            f"Righe valide: {righe_valide} | "
            f"Corretti: {corretti} | "
            f"Accuracy: {accuracy_temp:.4f} | "
            f"Formati sbagliati: {formati_sbagliati}"
        )                                      # Stampa metriche intermedie di valutazione

# Costruisci il contenuto da scrivere
risultati_del_test = (
    "*" * 10 + "\n" +
    f"Esito (Accuracy): {corretti / righe_valide if righe_valide > 0 else 0:.4f}\n\n" +
    f"Predizioni Corrette: {corretti}\n\n" +
    f"Predizioni Totali: {righe_valide}\n\n" +
    f"Numero di Formati Sbagliati: {risposte.count('Formato Sbagliato')}\n" +
    "*" * 10
)

# File di output per salvare i risultati
output_file = r"C:\\Users\\corra\\Desktop\\università\\AISE\\progetto\\Argus\\risultati_test.txt"

# Scrivi su file
with open(output_file, "w", encoding="utf-8") as f:
    f.write(risultati_del_test)

print(f"Riepilogo scritto su file: {output_file}")



In [ ]:
def confronta_liste(lista1, lista2):            # Definisce una funzione per confrontare due liste elemento per elemento
    indici_diversi = []                         # Lista che conterrà gli indici in cui le due liste differiscono

    max_len = max(len(lista1), len(lista2))     # Calcola la lunghezza massima tra le due liste
    for i in range(max_len):                    # Itera su tutti gli indici fino alla lunghezza massima
        if i >= len(lista1):
            indici_diversi.append(i)            # Aggiunge l’indice se l’elemento manca in lista1
        elif i >= len(lista2):
            indici_diversi.append(i)            # Aggiunge l’indice se l’elemento manca in lista2
        elif lista1[i] != lista2[i]:
            indici_diversi.append(i)            # Aggiunge l’indice se gli elementi sono diversi

    return indici_diversi                       # Restituisce la lista degli indici con differenze

Indici = confronta_liste(risposte, risposte_vere)  # Confronta predizioni e label reali e salva gli indici discordanti


In [ ]:
print(Indici)                                              # Stampa la lista degli indici in cui le predizioni differiscono dalle label reali

for indice in Indici:                                      # Itera su ciascun indice errato
    print(                                                 # Stampa il dettaglio dell’errore per l’indice corrente
        "Indice:", indice,
        " Risposte ", risposte[indice],
        " Risposta Corretta: ", risposte_vere[indice]
    )
